<a href="https://colab.research.google.com/github/R4HUL-ROY/Multimodal_feature_extraction/blob/main/glove_grid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import urllib.request
import numpy as np
import pandas as pd
import pathlib

In [2]:
df = pd.read_csv("/content/drive/MyDrive/project_resources/merged_data_for_multimodal_model.csv")
print(len(df))
df.head()

3482


,text_paths,texts,img_paths,data_label
0,/content/drive/MyDrive/tobaco_OCR/ADVE/0000136...,\n \n \n \n A Mpertant as yar\n sesiye teaeter...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,ADVE
1,/content/drive/MyDrive/tobaco_OCR/ADVE/0000435...,\n TE che fitm\n m66400 7127\n KOOLS are the o...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,ADVE
2,/content/drive/MyDrive/tobaco_OCR/ADVE/0030049...,SR Onrel ules cee\n Nee dss\n The one tales WT...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,ADVE
3,/content/drive/MyDrive/tobaco_OCR/ADVE/0000556...,so ARN Rr nr\n BWR Ga ||\n Vending Operators\n...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,ADVE
4,/content/drive/MyDrive/tobaco_OCR/ADVE/0349627...,\n &\n BR. :\n er non\n be 4\n op Re eo eee ee...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/03...,ADVE


In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['data_label']= le.fit_transform(df['data_label'])
df.head()

,text_paths,texts,img_paths,data_label
0,/content/drive/MyDrive/tobaco_OCR/ADVE/0000136...,\n \n \n \n A Mpertant as yar\n sesiye teaeter...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
1,/content/drive/MyDrive/tobaco_OCR/ADVE/0000435...,\n TE che fitm\n m66400 7127\n KOOLS are the o...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
2,/content/drive/MyDrive/tobaco_OCR/ADVE/0030049...,SR Onrel ules cee\n Nee dss\n The one tales WT...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
3,/content/drive/MyDrive/tobaco_OCR/ADVE/0000556...,so ARN Rr nr\n BWR Ga ||\n Vending Operators\n...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
4,/content/drive/MyDrive/tobaco_OCR/ADVE/0349627...,\n &\n BR. :\n er non\n be 4\n op Re eo eee ee...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/03...,0


In [4]:
label_dict = {}
for idx, row in enumerate(df['text_paths']):
    path = df.at[idx, 'text_paths']
    lab = path.split("/")[-2]
    label_dict[str(lab)] = df.at[idx, 'data_label']
    
print(label_dict)

{'ADVE': 0, 'Form': 2, 'Note': 6, 'Email': 1, 'News': 5, 'Resume': 8, 'Scientific': 9, 'Memo': 4, 'Report': 7, 'Letter': 3}


In [5]:
import re
def preprocess(text_string):
    preprocessed_string = re.sub(r'[^\w\s]','',text_string)
    preprocessed_string = preprocessed_string.replace('\n',' ')
    preprocessed_string = preprocessed_string.replace('_',' ')
    preprocessed_string = re.sub(' +', ' ', preprocessed_string)
    return preprocessed_string

In [6]:
# Tokenize, Lemmatize, stopwords removal
import spacy 
import nltk
# nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
nlp = spacy.load("en_core_web_sm")
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# stops = stopwords.words("english")

def get_stopwords(file_path):
    with open(file_path, "r") as fp:
        content = fp.read()
        stops = content.split("\n")
        stops = stops[:-1]
        fp.close()
        return stops

stops = get_stopwords("/content/drive/MyDrive/project_resources/stopwords.txt")
def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

normalize("counting playing the Home", lowercase=True, remove_stopwords=True)

'count play home'

In [7]:
df['texts'] = [preprocess(str(this_text)) for this_text in df['texts']]
df['texts'] = [normalize(this_text, lowercase=True, remove_stopwords=True) for this_text in df['texts']]
df.head()

,text_paths,texts,img_paths,data_label
0,/content/drive/MyDrive/tobaco_OCR/ADVE/0000136...,mpertant yar sesiye teaetere cabiieess baely k...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
1,/content/drive/MyDrive/tobaco_OCR/ADVE/0000435...,te che fitm m66400 7127 kool cigarette taste g...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
2,/content/drive/MyDrive/tobaco_OCR/ADVE/0030049...,sr onrel ules cee nee dss one tale wt lower ta...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
3,/content/drive/MyDrive/tobaco_OCR/ADVE/0000556...,arn rr nr bwr ga vend operator column worth 8 ...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
4,/content/drive/MyDrive/tobaco_OCR/ADVE/0349627...,br er non 4 op eo eee ee eee talk smoking deci...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/03...,0


In [8]:
from sklearn.model_selection import train_test_split
label = df['data_label']
X_train, X_dummy, y_train, y_dummy = train_test_split(df['texts'], df['data_label'] , test_size=0.2, random_state = 42)
X_test, X_val, y_test, y_val = train_test_split(X_dummy, y_dummy , test_size=0.5, random_state = 42)

In [9]:
print(len(X_train), len(y_train), len(X_test), len(y_test), len(X_val), len(y_val))

2785 2785 348 348 349 349


# Glove on train and test dataset seperately


In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_len = 500

In [11]:
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(X_train)

In [12]:
train_sequence = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequence, maxlen = max_len, truncating = "post", padding = "post" )

In [13]:
test_sequence = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequence, maxlen = max_len, truncating = "post", padding = "post" )

In [14]:
val_sequence = tokenizer.texts_to_sequences(X_val)
val_padded = pad_sequences(val_sequence, maxlen = max_len, truncating = "post", padding = "post" )

In [15]:
emmbed_dict = {}
with open('/content/drive/MyDrive/project_resources/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector
f.close()

In [16]:
word_index = tokenizer.word_index

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 200))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = emmbed_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [19]:
embedding_matrix.shape

(65837, 200)

In [24]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense,Flatten, Dropout, Conv1D, GlobalMaxPooling1D,GlobalAveragePooling1D, MaxPooling1D
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam


def create_model(filter, stride, kernel, num_layers, kernel_initializers, bias_initializers, dropout_rate, dropout_rate_conv, learning_rate):
    cnn = Sequential()
    cnn.add(
        Embedding(
            num_words,
            200,
            input_length = max_len,
            embeddings_initializer = Constant(embedding_matrix),
            trainable = False
        )
    )
    cnn.add(Dropout(dropout_rate))
    for i in range(num_layers):
        cnn.add(Conv1D(filters=filter,kernel_size=kernel,padding='valid',activation='relu',strides=stride, 
                       kernel_initializer=kernel_initializers,bias_initializer=bias_initializers))
        cnn.add(MaxPooling1D())
        cnn.add(Dropout(dropout_rate_conv))

    cnn.add(Flatten())    
    cnn.add(Dropout(dropout_rate_conv))

    cnn.add(Dense(128, activation = 'relu', kernel_initializer=kernel_initializers, bias_initializer=bias_initializers))
    cnn.add(Dense(10, activation='softmax'))
    optimizer = Adam(learning_rate = learning_rate)
    cnn.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics = ["accuracy"])
    return cnn

In [25]:
# def create_model(filter, stride, kernel, num_layers, kernel_initializers, bias_initializers, dropout_rate, dropout_rate_conv):
tuned_model = create_model(128, 1, 5, 4, "glorot_uniform", "zeros", 0.2, 0.5, 0.001)
history = tuned_model.fit(train_padded,y_train,epochs = 10,validation_data = (val_padded, y_val),verbose=2)

Epoch 1/10
88/88 - 3s - loss: 2.0451 - accuracy: 0.2679 - val_loss: 2.1251 - val_accuracy: 0.2493 - 3s/epoch - 38ms/step
Epoch 2/10
88/88 - 1s - loss: 1.8456 - accuracy: 0.3034 - val_loss: 1.9906 - val_accuracy: 0.2751 - 1s/epoch - 13ms/step
Epoch 3/10
88/88 - 1s - loss: 1.7227 - accuracy: 0.3551 - val_loss: 1.7131 - val_accuracy: 0.3553 - 1s/epoch - 13ms/step
Epoch 4/10
88/88 - 1s - loss: 1.5642 - accuracy: 0.4097 - val_loss: 1.6322 - val_accuracy: 0.4126 - 1s/epoch - 13ms/step
Epoch 5/10
88/88 - 1s - loss: 1.4371 - accuracy: 0.4596 - val_loss: 1.4387 - val_accuracy: 0.4441 - 1s/epoch - 13ms/step
Epoch 6/10
88/88 - 1s - loss: 1.3249 - accuracy: 0.4926 - val_loss: 1.3458 - val_accuracy: 0.4957 - 1s/epoch - 14ms/step
Epoch 7/10
88/88 - 1s - loss: 1.2226 - accuracy: 0.5235 - val_loss: 1.1903 - val_accuracy: 0.5358 - 983ms/epoch - 11ms/step
Epoch 8/10
88/88 - 1s - loss: 1.1786 - accuracy: 0.5490 - val_loss: 1.3700 - val_accuracy: 0.4756 - 983ms/epoch - 11ms/step
Epoch 9/10
88/88 - 1s - lo

In [26]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
model = KerasClassifier(build_fn=create_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
params={'filter':[128, 256], 
        'stride':[1,2],
        'kernel':[3,4,5],
        'num_layers' : [3,4],
        'kernel_initializers': ['glorot_uniform'],
        'bias_initializers':['zeros'],
        'dropout_rate' : [0.2],
        'dropout_rate_conv' : [0.5],
        'batch_size' : [64, 128],
        'epochs' : [20],
        'learning_rate' : [0.001]
        }
       
gs=GridSearchCV(estimator=model, param_grid=params, cv=5, verbose=3)
gs = gs.fit(train_padded, y_train)

Streaming output truncated to the last 5000 lines.
Epoch 8/20
35/35 [==============================] - 1s 15ms/step - loss: 1.0912 - accuracy: 0.6055
Epoch 9/20
35/35 [==============================] - 1s 15ms/step - loss: 1.0075 - accuracy: 0.6360
Epoch 10/20
35/35 [==============================] - 1s 16ms/step - loss: 0.9030 - accuracy: 0.6777
Epoch 11/20
35/35 [==============================] - 1s 15ms/step - loss: 0.8096 - accuracy: 0.7105
Epoch 12/20
35/35 [==============================] - 1s 15ms/step - loss: 0.7966 - accuracy: 0.7110
Epoch 13/20
35/35 [==============================] - 1s 15ms/step - loss: 0.7289 - accuracy: 0.7388
Epoch 14/20
35/35 [==============================] - 1s 15ms/step - loss: 0.6807 - accuracy: 0.7585
Epoch 15/20
35/35 [==============================] - 1s 15ms/step - loss: 0.5999 - accuracy: 0.7832
Epoch 16/20
35/35 [==============================] - 1s 15ms/step - loss: 0.5577 - accuracy: 0.7908
Epoch 17/20
35/35 [==============================] 

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 236, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.buil

44/44 [==============================] - 2s 30ms/step - loss: 1.9594 - accuracy: 0.2984
Epoch 2/20
44/44 [==============================] - 1s 26ms/step - loss: 1.7306 - accuracy: 0.3569
Epoch 3/20
44/44 [==============================] - 1s 26ms/step - loss: 1.4531 - accuracy: 0.4657
Epoch 4/20
44/44 [==============================] - 1s 26ms/step - loss: 1.2010 - accuracy: 0.5544
Epoch 5/20
44/44 [==============================] - 1s 26ms/step - loss: 1.0230 - accuracy: 0.6384
Epoch 6/20
44/44 [==============================] - 1s 26ms/step - loss: 0.9016 - accuracy: 0.6844
Epoch 7/20
44/44 [==============================] - 1s 26ms/step - loss: 0.8337 - accuracy: 0.7142
Epoch 8/20
44/44 [==============================] - 1s 26ms/step - loss: 0.7328 - accuracy: 0.7440
Epoch 9/20
44/44 [==============================] - 1s 26ms/step - loss: 0.7199 - accuracy: 0.7422
Epoch 10/20
44/44 [==============================] - 1s 26ms/step - loss: 0.6657 - accuracy: 0.7709
Epoch 11/20
44/44 [=

In [28]:
best_params = gs.best_params_
accuracy = gs.best_score_
print(best_params)
print(accuracy)

{'batch_size': 64, 'bias_initializers': 'zeros', 'dropout_rate': 0.2, 'dropout_rate_conv': 0.5, 'epochs': 20, 'filter': 256, 'kernel': 3, 'kernel_initializers': 'glorot_uniform', 'learning_rate': 0.001, 'num_layers': 3, 'stride': 1}
0.771633756160736


In [70]:
filter_size = best_params['filter']
kernels = best_params['kernel']
stride_unit = best_params['stride']
num_layers = best_params['num_layers']
kernel_initializer = best_params['kernel_initializers']
bias_initializer = best_params['bias_initializers']
dropout_rate_1 = best_params['dropout_rate_1']
dropout_rate_2 = best_params['dropout_rate_2']
batch_size = best_params['batch_size']
epochs = best_params['epochs']


# filter_size = 128
# kernels = 3
# stride_unit = 1
# print(filter_size)
tuned_model = create_model(filter_size, kernels, stride_unit, num_layers, kernel_initializer, bias_initializer)

In [25]:
history = tuned_model.fit(train_padded,y_train,epochs = 30,validation_data = (val_padded, y_val),verbose=2)

Epoch 1/30
88/88 - 3s - loss: 2.0597 - accuracy: 0.2589 - val_loss: 2.1226 - val_accuracy: 0.2865 - 3s/epoch - 34ms/step
Epoch 2/30
88/88 - 1s - loss: 1.8432 - accuracy: 0.3127 - val_loss: 2.0178 - val_accuracy: 0.2092 - 994ms/epoch - 11ms/step
Epoch 3/30
88/88 - 1s - loss: 1.7581 - accuracy: 0.3436 - val_loss: 1.8645 - val_accuracy: 0.3295 - 1s/epoch - 15ms/step
Epoch 4/30
88/88 - 1s - loss: 1.5970 - accuracy: 0.3996 - val_loss: 1.6038 - val_accuracy: 0.3811 - 883ms/epoch - 10ms/step
Epoch 5/30
88/88 - 1s - loss: 1.4731 - accuracy: 0.4521 - val_loss: 1.5370 - val_accuracy: 0.4527 - 707ms/epoch - 8ms/step
Epoch 6/30
88/88 - 1s - loss: 1.3851 - accuracy: 0.4833 - val_loss: 1.4221 - val_accuracy: 0.4842 - 700ms/epoch - 8ms/step
Epoch 7/30
88/88 - 1s - loss: 1.2683 - accuracy: 0.5174 - val_loss: 1.2801 - val_accuracy: 0.5473 - 700ms/epoch - 8ms/step
Epoch 8/30
88/88 - 1s - loss: 1.2106 - accuracy: 0.5422 - val_loss: 1.1866 - val_accuracy: 0.5903 - 699ms/epoch - 8ms/step
Epoch 9/30
88/88 -

In [106]:
tuned_model.evaluate(test_padded, y_test)

11/11 [==============================] - 0s 27ms/step - loss: 0.6322 - accuracy: 0.7672


[0.6322477459907532, 0.767241358757019]